# Chapter 28: Subcommands and Groups

This notebook covers advanced `argparse` patterns for building complex CLI tools: subparsers for git-style subcommands, mutually exclusive groups for conflicting options, and argument groups for organized help output.

## Key Concepts
- **Subparsers**: Create distinct subcommands (like `git init`, `git clone`)
- **Mutually exclusive groups**: Prevent conflicting options from being used together
- **Argument groups**: Organize related arguments in help output
- **Defaults per subcommand**: Each subcommand can have its own arguments and defaults

## Section 1: Subparsers — Git-Style Subcommands

Many CLI tools use subcommands: `git clone`, `docker run`, `pip install`. In `argparse`, this is achieved with `add_subparsers()`.

In [ ]:
import argparse

# Create the top-level parser
parser: argparse.ArgumentParser = argparse.ArgumentParser(prog="vcs")

# add_subparsers creates a special action for subcommands
# dest stores which subcommand was chosen
subparsers = parser.add_subparsers(dest="command", help="Available commands")

# Each subcommand gets its own parser
init_parser = subparsers.add_parser("init", help="Initialize a new repository")
init_parser.add_argument("--bare", action="store_true", help="Create a bare repository")

clone_parser = subparsers.add_parser("clone", help="Clone a repository")
clone_parser.add_argument("url", help="Repository URL to clone")
clone_parser.add_argument("--depth", type=int, help="Shallow clone depth")

parser.print_help()

In [ ]:
# Parsing the 'init' subcommand
args: argparse.Namespace = parser.parse_args(["init", "--bare"])

print(f"Command: {args.command}")
print(f"Bare: {args.bare}")
print(f"Full namespace: {vars(args)}")

In [ ]:
# Parsing the 'clone' subcommand
args = parser.parse_args(["clone", "https://example.com/repo", "--depth", "1"])

print(f"Command: {args.command}")
print(f"URL: {args.url}")
print(f"Depth: {args.depth}")

In [ ]:
# Subcommand-specific help
print("=== clone subcommand help ===")
clone_parser.print_help()

## Section 2: Dispatching Subcommands

A common pattern is using `set_defaults(func=...)` to associate each subcommand with a handler function. This avoids long if/elif chains.

In [ ]:
from typing import Any


def handle_start(args: argparse.Namespace) -> str:
    """Handle the 'start' subcommand."""
    return f"Starting service '{args.name}' on port {args.port}"


def handle_stop(args: argparse.Namespace) -> str:
    """Handle the 'stop' subcommand."""
    force_msg: str = " (forced)" if args.force else ""
    return f"Stopping service '{args.name}'{force_msg}"


def handle_status(args: argparse.Namespace) -> str:
    """Handle the 'status' subcommand."""
    return f"Status of all services (verbose={args.verbose})"


# Build parser with function dispatch
parser = argparse.ArgumentParser(prog="svc")
sub = parser.add_subparsers(dest="command")

# 'start' subcommand
start_cmd = sub.add_parser("start")
start_cmd.add_argument("name")
start_cmd.add_argument("--port", type=int, default=8080)
start_cmd.set_defaults(func=handle_start)

# 'stop' subcommand
stop_cmd = sub.add_parser("stop")
stop_cmd.add_argument("name")
stop_cmd.add_argument("--force", action="store_true")
stop_cmd.set_defaults(func=handle_stop)

# 'status' subcommand
status_cmd = sub.add_parser("status")
status_cmd.add_argument("-v", "--verbose", action="store_true")
status_cmd.set_defaults(func=handle_status)

# Dispatch: parse args and call the associated function
for cmd_line in ["start web --port 3000", "stop web --force", "status -v"]:
    args = parser.parse_args(cmd_line.split())
    result: str = args.func(args)
    print(f"  $ svc {cmd_line}")
    print(f"    -> {result}\n")

## Section 3: Mutually Exclusive Groups

Mutually exclusive groups prevent two options from being used at the same time. This is useful for output format flags or conflicting modes.

In [ ]:
# Mutually exclusive output format flags
parser = argparse.ArgumentParser(prog="export")
format_group = parser.add_mutually_exclusive_group()
format_group.add_argument("--json", action="store_true", help="Output as JSON")
format_group.add_argument("--csv", action="store_true", help="Output as CSV")
format_group.add_argument("--xml", action="store_true", help="Output as XML")

# Only one format flag can be used
args = parser.parse_args(["--json"])
print(f"json: {args.json}, csv: {args.csv}, xml: {args.xml}")

args = parser.parse_args(["--csv"])
print(f"json: {args.json}, csv: {args.csv}, xml: {args.xml}")

In [ ]:
# Attempting to use two mutually exclusive options
try:
    parser.parse_args(["--json", "--csv"])
except SystemExit:
    print("Error: --json and --csv cannot be used together")
    print("argparse raises SystemExit when mutually exclusive args conflict")

In [ ]:
# required=True forces the user to pick exactly one option
parser = argparse.ArgumentParser(prog="convert")
mode_group = parser.add_mutually_exclusive_group(required=True)
mode_group.add_argument("--encode", action="store_true", help="Encode the input")
mode_group.add_argument("--decode", action="store_true", help="Decode the input")

parser.add_argument("input_file", help="File to process")

args = parser.parse_args(["--encode", "data.bin"])
print(f"encode: {args.encode}, decode: {args.decode}, file: {args.input_file}")

# Omitting both from a required group is an error
try:
    parser.parse_args(["data.bin"])
except SystemExit:
    print("\nError: one of --encode or --decode is required")

## Section 4: Determining the Selected Format

When using mutually exclusive boolean flags, a helper function can determine which was selected.

In [ ]:
def get_output_format(args: argparse.Namespace) -> str:
    """Determine the selected output format from mutually exclusive flags."""
    if args.json:
        return "json"
    elif args.csv:
        return "csv"
    elif args.xml:
        return "xml"
    return "text"  # Default fallback


parser = argparse.ArgumentParser(prog="report")
fmt_group = parser.add_mutually_exclusive_group()
fmt_group.add_argument("--json", action="store_true")
fmt_group.add_argument("--csv", action="store_true")
fmt_group.add_argument("--xml", action="store_true")

# Test each format
for flags in [["--json"], ["--csv"], ["--xml"], []]:
    args = parser.parse_args(flags)
    fmt: str = get_output_format(args)
    print(f"Flags: {flags or ['(none)']} -> format: {fmt}")

## Section 5: Argument Groups

Argument groups organize arguments into logical sections in the help output. Unlike mutually exclusive groups, they do not restrict which arguments can be used together.

In [ ]:
# Argument groups for organized help output
parser = argparse.ArgumentParser(prog="deploy", description="Deploy an application")

# Connection arguments
conn_group = parser.add_argument_group("connection", "Server connection options")
conn_group.add_argument("--host", default="localhost", help="Server hostname")
conn_group.add_argument("--port", type=int, default=22, help="Server port")
conn_group.add_argument("--user", default="deploy", help="SSH username")

# Deployment arguments
deploy_group = parser.add_argument_group("deployment", "Deployment configuration")
deploy_group.add_argument("--branch", default="main", help="Git branch to deploy")
deploy_group.add_argument("--tag", help="Specific tag to deploy")
deploy_group.add_argument("--rollback", action="store_true", help="Rollback to previous version")

# Notification arguments
notify_group = parser.add_argument_group("notifications", "Notification settings")
notify_group.add_argument("--notify-slack", action="store_true", help="Send Slack notification")
notify_group.add_argument("--notify-email", help="Send email notification to address")

parser.print_help()

In [ ]:
# All groups can be used simultaneously (they are just for organization)
args = parser.parse_args([
    "--host", "prod.example.com",
    "--port", "2222",
    "--branch", "release/v2",
    "--notify-slack",
])

print(f"Host: {args.host}:{args.port}")
print(f"User: {args.user}")
print(f"Branch: {args.branch}")
print(f"Notify Slack: {args.notify_slack}")

## Section 6: Combining Subcommands with Groups

A real-world CLI often combines subparsers with argument groups and mutually exclusive options within each subcommand.

In [ ]:
def build_db_cli() -> argparse.ArgumentParser:
    """Build a CLI parser for a database management tool."""
    parser = argparse.ArgumentParser(
        prog="dbctl",
        description="Database management tool",
    )
    parser.add_argument("-v", "--verbose", action="store_true")

    sub = parser.add_subparsers(dest="command")

    # 'migrate' subcommand with mutually exclusive direction
    migrate = sub.add_parser("migrate", help="Run database migrations")
    direction = migrate.add_mutually_exclusive_group(required=True)
    direction.add_argument("--up", action="store_true", help="Apply migrations")
    direction.add_argument("--down", action="store_true", help="Rollback migrations")
    migrate.add_argument("--steps", type=int, default=1, help="Number of steps")

    # 'dump' subcommand with argument groups
    dump = sub.add_parser("dump", help="Export database")

    source_group = dump.add_argument_group("source", "Database source options")
    source_group.add_argument("--database", required=True, help="Database name")
    source_group.add_argument("--tables", nargs="*", help="Specific tables to dump")

    output_group = dump.add_argument_group("output", "Output options")
    fmt = output_group.add_mutually_exclusive_group()
    fmt.add_argument("--sql", action="store_true", help="SQL format")
    fmt.add_argument("--csv", action="store_true", help="CSV format")
    output_group.add_argument("-o", "--output", default="stdout", help="Output file")

    return parser


db_cli: argparse.ArgumentParser = build_db_cli()
db_cli.print_help()

In [ ]:
# Using the migrate subcommand
args = db_cli.parse_args(["migrate", "--up", "--steps", "3"])
print(f"Command: {args.command}")
print(f"Direction: {'up' if args.up else 'down'}")
print(f"Steps: {args.steps}")

print()

# Using the dump subcommand with groups
args = db_cli.parse_args([
    "-v", "dump",
    "--database", "mydb",
    "--tables", "users", "orders",
    "--csv",
    "-o", "export.csv",
])
print(f"Command: {args.command}")
print(f"Verbose: {args.verbose}")
print(f"Database: {args.database}")
print(f"Tables: {args.tables}")
print(f"CSV format: {args.csv}")
print(f"Output: {args.output}")

## Section 7: Parent Parsers for Shared Arguments

Parent parsers let you define common arguments once and share them across subcommands without repeating definitions.

In [ ]:
# Shared parent parser with common arguments
parent = argparse.ArgumentParser(add_help=False)  # add_help=False avoids duplicate --help
parent.add_argument("--debug", action="store_true", help="Enable debug mode")
parent.add_argument("--config", default="config.toml", help="Config file path")

# Main parser uses the parent
main_parser = argparse.ArgumentParser(prog="app")
sub = main_parser.add_subparsers(dest="command")

# Both subcommands inherit --debug and --config from the parent
serve_cmd = sub.add_parser("serve", parents=[parent], help="Start the server")
serve_cmd.add_argument("--port", type=int, default=8080)

test_cmd = sub.add_parser("test", parents=[parent], help="Run tests")
test_cmd.add_argument("--coverage", action="store_true")

# Both subcommands have --debug and --config
args = main_parser.parse_args(["serve", "--debug", "--port", "3000"])
print(f"serve: debug={args.debug}, config={args.config}, port={args.port}")

args = main_parser.parse_args(["test", "--debug", "--coverage"])
print(f"test: debug={args.debug}, config={args.config}, coverage={args.coverage}")

## Summary

### Subparsers
- **`add_subparsers(dest=...)`** creates a subcommand dispatcher
- **`add_parser(name)`** registers a new subcommand with its own arguments
- **`set_defaults(func=...)`** enables clean function dispatch per subcommand

### Mutually Exclusive Groups
- **`add_mutually_exclusive_group()`** prevents conflicting flags from being used together
- Use `required=True` to force exactly one choice from the group
- Argparse raises `SystemExit` if conflicting options are provided

### Argument Groups
- **`add_argument_group(title, description)`** organizes help output into sections
- Groups are purely organizational and do not restrict argument usage
- Combine groups within subcommands for well-structured CLI tools

### Parent Parsers
- **`parents=[parent_parser]`** shares common arguments across subcommands
- Use `add_help=False` on parent parsers to avoid duplicate `--help` flags